parser:
Where we need more BB?
1- if-else
2- while
3- for

In [23]:
class parser_util():
    def __init__(self):
        pass
    
    def check_computation(self):
        self.must_parse("main")
        #more than 0
        while self.tokens[0].type in ["var","array"]:
            self.check_varDecl()
        #more than 0
        while self.tokens[0].type in ["function","procedure"]:
            self.check_funcDecl()
        self.must_parse("OPEN_C")        
        self.check_stat_Sequence(0)
        self.must_parse("CLOSE_C")
        self.must_parse("POINT")
        
    def check_funcDecl(self):
        if self.tokens[0].type=="function":
            self.must_parse("function")
        else:
            self.must_parse("procedure")
        self.must_parse("INDENT")
        self.check_formalParam()
        self.must_parse("SEMI")
        self.check_funcBody()
        self.must_parse("SEMI")
        
    def check_formalParam(self):
        if self.tokens[0].type=="OPEN_P":
            self.must_parse("OPEN_P")
            if self.tokens[0].type=="INDENT":
                self.must_parse("INDENT")
                #more than 0
                while self.tokens[0].type=="COMMA":            
                    self.must_parse("COMMA")
                    self.must_parse("INDENT")
            self.must_parse("CLOSE_P")
         
    def check_funcBody(self):
        #more than 0
        bb=0
        while self.tokens[0].type in ["var","array"]:
            self.check_varDecl()
        self.must_parse("OPEN_C")        
        self.check_stat_Sequence(bb)
        self.must_parse("CLOSE_C")

    def check_relation(self,bb):
        self.check_expression(bb)
        self.check_relOp()
        self.check_expression(bb)
        
    def check_relOp(self):
        if self.tokens[0].type in ['OP_EQ','OP_NEQ','OP_LESS','OP_LEQ','OP_GRT','OP_GEREQ']:
            self.must_parse(self.tokens[0].type)
        
    def must_parse(self,target_token):        
        sym=self.tokens.popleft()
        if (sym.type==target_token):
            if sym.type=="INDENT":
                print(sym.value),
            pass
        else:
            print("error!!!!!",target_token,sym,inspect.stack()[1][3])
            print([i.value for i in self.tokens])  


            
class Result():
    def __init__(self, kind=None,val=None,address=None,regno=None):
        self.kind = kind
        self.val = val
        self.address = address
        self.regno=regno
    def __repr__(self):
        return "$"+str(self.kind) + str(self.val) + str(self.address) + str(self.regno) + "$"

    
class BB():
    def __init__(self):
        self.node=defaultdict()
        self.node[0]=[[],[]]
        self.number=0
    
    def new_bb(self,parents):
        self.number=self.number+1
        print("made new BB",self.number)
        self.node[self.number]=[[],[]]
        for p in parents:
            self.node[p][0].append(self.number)
        return self.number
    def visulize(self):
        pass
    
    def print_bb(self):
        print(self.node.keys())
        for i in self.node.keys():
            print("====BB=====",i)
            for j in self.node[i][1]:
                print(j)
        for i in self.node.keys():
            print("bb",i,"connect to bb:",self.node[i][0])

In [27]:
import collections 
import inspect
%run tokenizer.ipynb
from collections import defaultdict
pair={"[":"]","{":"}","(":")"}

opCode={"+":"add","-":"sub","*":"mul","\\":"div"}
        
class Reg():
    def __init__(self):
        self.num=5
        self.occ= [0]*self.num 
        self.cfg=BB()

        
    def AllocateReg(self):
        for i in range(0,self.num):
            if self.occ[i]==0:
                self.occ[i]=1
                print(self.occ)
                return i
        print("not enough registers!!!!")
        return -1
    def DeAllocateReg(self,i):
        if i>=0 and i<self.num:
            self.occ[i]=0
        else:
            print("register deallocated out of range!!!!",i)
        print(self.occ)
        
    #4 values: op,target:x.regno, a.regno,b.regno
    def emit(self,bb,op,x=None,y=None):
        if y is not None:
            line=op +" " +str(x.regno) +","+ str(y.regno)
        else:
            line=op +" " +str(x.regno)
        self.cfg[bb][1].append(line)
    
    #4 values: op,target:x.regno, a.regno,b.regno
    def emit1(self,bb,op,s):
        line=op +" " +s
        #print(self.cfg.node[bb])
        self.cfg.node[bb][1].append(line)
        
    def load(self,x,bb):
        if x.kind=='var':
            x.regno=self.AllocateReg()
            #TODO: emit
            self.emit1(bb,"LDW",str(x.regno)+str(0)+str(0))
            x.kind='var'
        elif x.kind=="constant":
            if x.value==0:
                x.regno=0
            else:
                x.regno=self.AllocateReg()
                #TODO: emit
                self.emit1(bb,"ADDI",str(x.regno)+str(0)+str(0))                
            x.kind='reg'
        return x
    
    def compute(self,x,y,op,bb):
        if x.kind=='constant' and y.kind=='constant':
            x.val= eval(str(x.val) + op + str(y.val))
        else:
            self.load(x,bb)
            if x.regno==0:
                x.regno=self.AllocateReg()
                #TODO: emit
                self.emit1(bb,"ADD",str(x.regno)+str(0)+str(0))                
            if y.kind=='constant':
                #TODO: emit
                self.emit1(bb,op,str(x.regno)+str(x.regno)+str(y.val))
            else:
                y=self.load(y,bb)
                #TODO: emit
                self.emit1(bb,op,str(x.regno)+str(x.regno)+str(y.regno))
                self.DeAllocateReg(y.regno)
                pass
        print("compute",x)
        return x

class SSA():
    def __init__(self):
        self.SSA_current_line=0
        self.variable_table=defaultdict(list)

    
class parser(SSA,Reg,parser_util,BB):
    def __init__(self,tokens,rules):
        self.tokens=collections.deque([i for i in tokens if i.type!='COMMENT'])
        self.rules=rules
        self.current_bb=0
        SSA.__init__(self)
        Reg.__init__(self)
        parser_util.__init__(self)
        BB.__init__(self)
 
        #for i in self.tokens:
        #    print(i.type)
    
    def check_stat_Sequence(self,bb):
        print("check_stat_Sequence:",bb,inspect.stack()[1][3])
        bb=self.check_statement(bb)
        # more that 0
        while self.tokens[0].type in ["SEMI"]:
            self.must_parse("SEMI")            
            bb=self.check_statement(bb)
            
    def check_statement(self,bb):
        if self.tokens[0].type not in ["let","call","if","while","return"]:
            print("error check_statement",self.tokens[0].type,inspect.stack()[1][3])
        if self.tokens[0].type=="let":
            self.check_assignment(bb)
        if self.tokens[0].type=="call":
            self.check_funcCall()
        if self.tokens[0].type=="if":
            bb=self.check_ifStatement(bb)
        if self.tokens[0].type=="while":
            self.check_whileStatement()
        if self.tokens[0].type=="return":
            self.check_returnStatement()
        return bb
    
    def check_returnStatement(self,bb):
        self.must_parse("return")
        #how?
        if self.tokens[0].type in ["INDENT","NUMBER","OPEN_P","call"]:
            self.check_expression(bb)

    def check_whileStatement(self):
        self.must_parse("while")
        self.check_relation(bb)
        self.must_parse("do")
        self.check_stat_Sequence()
        self.must_parse("od")
        
    
    def check_ifStatement(self,bb):
        self.must_parse("if")
        self.check_relation(bb)
        
        self.must_parse("then")
        newbbif=self.cfg.new_bb([bb])        
        self.check_stat_Sequence(newbbif)        
        
        next_bb=[newbbif]
        
        if self.tokens[0].type=="else":
            self.must_parse("else")
            newbbelse=self.cfg.new_bb([bb])
            self.check_stat_Sequence(newbbelse)
            next_bb.append(newbbelse)

        self.must_parse("fi")
        bb=self.cfg.new_bb(next_bb)
        return bb    
    
    def check_assignment(self,bb):
        self.must_parse("let")
        self.check_designator(bb)
        self.must_parse("ASSIGN")
        self.check_expression(bb)
    
    def check_designator(self,bb):
        #self.cfg.node[bb][1].append(line)
        #self.SSA.variable_table[self.tokens[0].val].append(self.SSA_current_line)
        self.must_parse("INDENT")        
        while self.tokens[0].type=="OPEN_B":
            self.must_parse("OPEN_B")
            self.check_expression(bb)
            self.must_parse("CLOSE_B")
    
    def check_funcCall(self):
        bb=0
        self.must_parse("call")
        self.must_parse("INDENT")
        if self.tokens[0].type=="OPEN_P":
            self.must_parse("OPEN_P")
            #how?
            if self.tokens[0].type in ["INDENT","NUMBER","OPEN_P","call"]:
                self.check_expression(bb)
                while self.tokens[0].type=="COMMA":
                        self.must_parse("COMMA")
                        self.check_expression(bb)
            self.must_parse("CLOSE_P")
        
    def check_varDecl(self):
        self.check_typeDecl()
        #print(self.tokens[0].value)
        self.must_parse("INDENT")
        #more than 0
        while self.tokens[0].type in ["COMMA"]:
            self.must_parse("COMMA")            
            #print(self.tokens[0].value)
            self.must_parse("INDENT")
        self.must_parse("SEMI")
        
    def check_typeDecl(self):
        if self.tokens[0].type=="array":
            self.must_parse("array")
            counter=0
            #more than 1
            while self.tokens[0].type=="OPEN_B":
                self.must_parse("OPEN_B")
                self.must_parse("NUMBER")
                self.must_parse("CLOSE_B")
                counter+=1
            if counter==0:
                print("error typeDecl",self.tokens[0].type)
                
        elif self.tokens[0].type=="var":
            self.must_parse("var")
        else:
            print("error typeDecl",target_token,self.tokens[0].type)
    
    def check_factor(self,bb):
        result=Result(None,None,None,None)
        if self.tokens[0].type not in ["INDENT","NUMBER","OPEN_P","call"]:
            print("error,check_factor!!!",self.tokens[0],inspect.stack()[1][3])
        if self.tokens[0].type=="INDENT":
            #TODO:what about array?
            result.kind,result.address='var',self.tokens[0].value
            self.check_designator(bb)              
        if self.tokens[0].type=="NUMBER":
            result.kind,result.val='constant',self.tokens[0].value 
            self.must_parse("NUMBER")            
        if self.tokens[0].type=="OPEN_P":
            self.must_parse("OPEN_P")
            result=self.check_expression(bb)
            self.must_parse("CLOSE_P")
        if self.tokens[0].type=="call":
            self.check_funcCall()
        print(result)
        return result
            
    
    def check_expression(self,bb):
        result_x=self.check_term(bb)
        while self.tokens[0].value in ["+","-"]:
            op=self.tokens[0].value
            if self.tokens[0].value=="+":
                self.must_parse("OP")
            if self.tokens[0].value=="-":
                self.must_parse("OP")
            result_y=self.check_term(bb)
            result_x=self.compute(result_x,result_y,op,bb)            
        return result_x
    
    def check_term(self,bb):
        result_x=self.check_factor(bb)
        while self.tokens[0].value in ["*","/"]:
            op=self.tokens[0].value
            if self.tokens[0].value=="*":
                self.must_parse("OP")
            if self.tokens[0].value=="/":
                self.must_parse("OP")
            result_y=self.check_factor(bb)
            result_x=self.compute(result_x,result_y,op,bb)
        return result_x
    
    def print_bb1(self):
        print(self.cfg.node.keys())
        for i in self.cfg.node.keys():
            print("====BB=====",i)
            for j in self.cfg.node[i][1]:
                print(j)
        for i in self.cfg.node.keys():
            print("bb",i,"connect to bb:",self.cfg.node[i][0])

In [28]:
tk=Tokenizer(l_kewords,token_specification)

code='''
# Nested if/while v3
# If I think I pass this, I'm moving on.
# This test should not be attempted by anyone who is pregnant, nursing, has 
# high blood pressure, aliens, and stressed out graduate students.
# Based on test 13.nt_bb
main
var x, y;
var a, b;
{
    if x > 1 then let y<-y+5 else let y<-y+5 fi;
    let x <- y + x
}
.
'''
tk.tokenize_program(code)

c=parser(tk.result,rules)
c.check_computation()
c.print_bb()

0 1 2 3 ('check_stat_Sequence:', 0, 'check_computation')
0 $varNone0None$
$constant1NoneNone$
('made new BB', 1)
('check_stat_Sequence:', 1, 'check_ifStatement')
1 1 $varNone1None$
$constant5NoneNone$
[1, 0, 0, 0, 0]
[1, 1, 0, 0, 0]
('compute', $varNone11$)
('made new BB', 2)
('check_stat_Sequence:', 2, 'check_ifStatement')
1 1 $varNone1None$
$constant5NoneNone$
[1, 1, 1, 0, 0]
('compute', $varNone12$)
('made new BB', 3)
0 1 $varNone1None$
0 $varNone0None$
[1, 1, 1, 1, 0]
[1, 1, 1, 1, 1]
[1, 1, 1, 1, 0]
('compute', $varNone13$)
[0]
('====BB=====', 0)
('bb', 0, 'connect to bb:', [])


import os
path="/home/ahmad/Documents/compiler/project/testprogs/"
for files in os.listdir(path):
    print(files,"==========")
    a="".join([lines for lines in open(path+files,'r')])
    tk=Tokenizer(l_kewords,token_specification)
    tk.tokenize_program(a)
    c=parser(tk.result,rules)
    c.check_computation("computation")
    #tk.tokenize_program(code)